In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

In [2]:
def create_search(keyword):
    cookies = {
        'advanced-frontend': '1qvup17bdkenkns7ja4ae37m0h',
        '_csrf-frontend': '336089dc6274e73228ccf35d519826f5d3bf03f31b26791424cedbd04e5f4472a%3A2%3A%7Bi%3A0%3Bs%3A14%3A%22_csrf-frontend%22%3Bi%3A1%3Bs%3A32%3A%22Veb1gE9jjqODCQlIEHn7wwZA7shoodp3%22%3B%7D',
        '_gid': 'GA1.3.807119358.1683069145',
        '_ga': 'GA1.3.67003413.1663762523',
        '_ga_3WFF0VD4BK': 'GS1.1.1683069145.12.1.1683069536.0.0.0',
    }

    headers = {
        'authority': 'mdar.mda.gov.my',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'max-age=0',
        # 'cookie': 'advanced-frontend=1qvup17bdkenkns7ja4ae37m0h; _csrf-frontend=336089dc6274e73228ccf35d519826f5d3bf03f31b26791424cedbd04e5f4472a%3A2%3A%7Bi%3A0%3Bs%3A14%3A%22_csrf-frontend%22%3Bi%3A1%3Bs%3A32%3A%22Veb1gE9jjqODCQlIEHn7wwZA7shoodp3%22%3B%7D; _gid=GA1.3.807119358.1683069145; _ga=GA1.3.67003413.1663762523; _ga_3WFF0VD4BK=GS1.1.1683069145.12.1.1683069536.0.0.0',
        'referer': 'https://mdar.mda.gov.my/frontend/web/index.php?r=carian',
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    }

    params = {
        'r': 'carian/index',
        'CompletedApplicationAllmdrSearch[globalSearch]': keyword,
    }

    response = requests.get('https://mdar.mda.gov.my/frontend/web/index.php', params=params, cookies=cookies, headers=headers)
    soup = BeautifulSoup(response.text)
    device_list = soup.find_all('tr')
    return device_list
    

In [3]:
# #Load Keyword file
# keyword_file = pd.ExcelFile("IVD_Keyword_Tracker_v0.1.xlsx")
# player_names = pd.read_excel(keyword_file, sheet_name = 'IVD Final Keyword List')
# player_names = set(player_names[player_names['Player Name (2)'].notnull()]['Player Name (2)'])
# player_names

{'Abbott',
 'Agilent',
 'BD',
 'BIO-RAD',
 'BIOMERIEUX',
 'Cepheid',
 'Diasorin',
 'Grifols',
 'Hologic',
 'Horiba',
 'Human',
 'Leica',
 'Merck',
 'Mindray',
 'Qiagen',
 'Roche',
 'Siemens',
 'Sysmex',
 'Thermo Fisher'}

In [3]:
## Use IVD Gateway Major player to search MDA using company names
major_players = pd.read_excel("IVD_gateway_major_players_actual.xlsx")
major_players 

,production_cycle,study_year,player_code,player_name
0,2023,2022,PY0196,Roche
1,2023,2022,PY0006,Abbott
2,2023,2022,PY0064,Danaher - Beckman Coulter
3,2023,2022,PY0065,Danaher - Cepheid
4,2023,2022,PY0066,Danaher - Leica
...,...,...,...,...
327,2023,2022,PY9995,Boule Diagnostics
328,2023,2022,PY0375,Fokus Diagnostic
329,2023,2022,PY0397,Bioptik Technology
330,2023,2022,PY0338,Inbios


In [4]:
players = major_players.player_name.values.tolist()
## Split out company names delimited by '-'
keywords = []
for name in players:
    if '-' in name:
        spread_names = name.split('-')
        for i in spread_names:
            keywords.append(i.strip())
    else:
        keywords.append(name)

In [5]:
len(set(keywords))
keywords = list(set(keywords))

In [7]:
print("Searching MDA using keywords...")
catch = dict()
for keyword in keywords:
    print(keyword, end=" ")
    try:
        search_result = create_search(keyword)
        search_result = search_result[1:]
        if 'No results found.' in search_result[0].text:
            print("No Results")
        else:
            ids_in_results = [x['data-key'] for x in search_result]
            catch[keyword] =  ids_in_results
            print(len(ids_in_results))
    except:
        print("Error")
        
    
    

Searching MDA using keywords...
Oxoid 4
Optica 46
Biochem (Lang Mai) No Results
Biosys 61
Turk Lab No Results
OptraScan No Results
Cytognos, S.L. No Results
Dakewe 11
Sens 251
Tulip 3
Biocartis No Results
Wama No Results
Hamilton 14
BioPerfectus 5
Diagnostic Automation / Cortez Diagnostics (DACD) No Results
Dymind 8
Wuhan Easy Diagnostics No Results
Immuno concepts 6
Diagast 23
Burning Rock No Results
GC Pharma No Results
Trek Diagnostic No Results
MBL 30
Orient Gene Biotech No Results
ECO Diagnóstica No Results
VWR No Results
Thermo Fisher Scientific 16
Kyokuto No Results
Yourgene No Results
Techno Medica No Results
Illumina 22
Hangzhou Testsea Biotechnology Co Ltd (China) No Results
Aldrich 2
TaiDoc 1
Medite 119
Roche 763
SD Biosensor 17
3 B Blackbio No Results
V Error
Tokyo Boeki 1
Fokus Diagnostic No Results
Werfen 1
Snibe 47
Bioneer 3
VivaChek Biotech (Hangzhou) Co.,Ltd No Results
D.S. Pharma No Results
ABI 325
Duksan No Results
Check 55
AmoyDx No Results
Da An 1
Osang No Results


In [8]:
catch

{'Oxoid': ['20870', '52826', '52954', '54126'],
 'Optica': ['19632',
  '21823',
  '22691',
  '23491',
  '23682',
  '23683',
  '24323',
  '24374',
  '33360',
  '33687',
  '35960',
  '37600',
  '40458',
  '48059',
  '52875',
  '54937',
  '56135',
  '58502',
  '60190',
  '61453',
  '66060',
  '66518',
  '66912',
  '67469',
  '67618',
  '69440',
  '70615',
  '71402',
  '73402',
  '73908',
  '75355',
  '79525',
  '82112',
  '84793',
  '85304',
  '85618',
  '87892',
  '88048',
  '88898',
  '89030',
  '89359',
  '89447',
  '90131',
  '90732',
  '90776',
  '91632'],
 'Biosys': ['22556',
  '22643',
  '36303',
  '37606',
  '38590',
  '39969',
  '49676',
  '50017',
  '56477',
  '57769',
  '59825',
  '60505',
  '60627',
  '60878',
  '61106',
  '61350',
  '61602',
  '62320',
  '62710',
  '64796',
  '65194',
  '66848',
  '67349',
  '68258',
  '68553',
  '68769',
  '69584',
  '69655',
  '69956',
  '70968',
  '71004',
  '71110',
  '71189',
  '73939',
  '74028',
  '74442',
  '74839',
  '76188',
  '7637

In [9]:
ids_to_scrape = []
for k,v in catch.items():
    ids_to_scrape+= [*v]

ids_to_scrape  = list(set(ids_to_scrape))
len(ids_to_scrape)
    

17734

In [ ]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
url = "https://mdar.mda.gov.my/frontend/web/index.php?r=carian%2Fview&id="
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})
print("Running")
errors = []
for ids in ids_to_scrape:
    try:
        response = requests.get(url+ids, headers = headers)
        time.sleep(1)

        soup = BeautifulSoup(response.text)
        trs = soup.find('table', id="w0").find_all('tr')
        for tr in trs:
            th = tr.find('th').text
            td = tr.find('td').text

            if th == "NAME OF DEVICE":
                name = td
            elif th == "ESTABLISHMENT NAME":
                company = td
            elif th == "REGISTRATION NO":
                rnumber = td
            elif th == "BRAND NAME":
                product = td
            elif th == "DESCRIPTION":
                desc = td
            elif th == "INTENDED PURPOSE":
                meta = td
            elif th == "VALIDITY DATE OF REGISTRATION":
                start = datetime.strptime(td.split(" - ")[0], '%d-%m-%Y')
                end = datetime.strptime(td.split(" - ")[1], '%d-%m-%Y')
        sku_data = soup.find
        row = [ids, rnumber, name, company, product, desc, meta, start, end, url]
        temp = pd.DataFrame(row)
        temp = temp.transpose()
        df1 = pd.concat([df1,temp])
        trs = soup.find('table', id="example2").find_all('tr')
        if len(trs) > 1:
            row1 = []
            for tr in trs[1:]:
                tds = tr.find_all('td')
                prod = tds[1].text
                identifier = tds[2].text
                row = [ids, prod, identifier]
                temp = pd.DataFrame(row)
                temp = temp.transpose()
                df2 = pd.concat([df2,temp])
    except:
        errors.append(ids)


Running


In [ ]:
import pickle
## store the dict for later.
with open(f'MDA_MY_MDR_Company_Records_{datetime.date.now().strftime("%Y%m%d")}.pkl', 'wb') as f:
    pickle.dump(catch, f)


In [ ]:
df1.to_csv("MDA_MY - Device list_110124.csv")
df2.to_csv("MDA_MY - Device Identifiers_110124.csv")

In [28]:
errors_d = pd.DataFrame(errors)
errors_d.to_excel("Errors.xlsx")